In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

In [ ]:
# alldf = pd.read_csv('state.csv',low_memory=False)
alldf = pd.read_parquet(os.path.join(hndl.sandbox_dir,'state.parquet'))
alldf.date = pd.to_datetime(alldf.date)
statename = alldf.bgStateName.iloc[0]

In [ ]:
# print( th.getMoleculeImg(cas,size=300))
showHeader(statename.title(), link_up_level=1)

In [ ]:
def xlate_val(n):
    if n==0:
        return ''
    if n<1000:
        return round_sig(n,1)
    x = round_sig(n,1)
    return x[0]+ 'k'

def make_annot(gb):
    annot = gb.copy()
    annot.DisclosureId = annot.DisclosureId.map(lambda x: xlate_val(x))
    #print(annot)
    piv = annot.pivot(index='County',columns='year',values='DisclosureId')
    piv.fillna('',inplace=True)
    #print(piv)
    return piv

def get_state_center(state):
    t = pd.read_csv(r"C:\MyDocs\OpenFF\src\openFF-catalog\work\state_coords.csv",
                   dtype={'Latitude':'float', 'Longitude':'float'})
    t = t[t.state==state]
    #print(t)
    return [t.Latitude.mean(),t.Longitude.mean()*-1]

    
def CountyMap(df):
    start_loc = get_state_center(statename)
    #print(statename,start_loc)
    cond = ~df.location_error
    if cond.sum()==0:  # no valid fracks for this state
        display(md(f'### No mappable fracks for {statename}!'))
        display(md(f'This is usually caused when "CountyName" in FracFocus dont match standard names'))
        return
    gb = df[cond].groupby(['bgStateName','bgCountyName',
                                                   'DisclosureId'],as_index=False).size()
    gb = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['DisclosureId'].count().rename({'bgStateName':'StateName',
                                                                                                'bgCountyName':'CountyName',
                                                                                                'DisclosureId':'value'},
                                                                                                axis=1)
    zoom = 6
    if statename in ['texas','california']:
        zoom = 5
    if statename in ['alaska']:
        zoom = 4
        
    mapping.create_county_choropleth(gb,plotlog=True,custom_scale= [0,1,2,3,4,5],
                             start_loc=start_loc, # center of state's data
                             legend_name='Number of FracFocus disclosures',
                             start_zoom=zoom,fields=['StateName','CountyName','orig_value'],
                             aliases=['State: ','County: ','Number Fracking disclosures: '])

def CountyCntTable(df):
    # first, make the general searchable table
    gb = df.groupby(['bgCountyName','DisclosureId'],as_index=False)['date'].first()
    gb['year'] = gb.date.dt.year.astype('str')
    gb1 = gb.groupby(['bgCountyName'],as_index=False)['DisclosureId'].count().rename({'DisclosureId':'disclosure_count'},
                                                                                  axis=1)
    gb2 = gb1.copy()
    gbop = df.groupby('bgCountyName')['OperatorName'].agg(lambda x:x.value_counts().index[0:4]).reset_index()
    gbop.OperatorName = gbop.OperatorName.map(lambda x: th.xlate_to_str(x,'; ',sort=False))
    gbop = gbop.rename({'OperatorName':'Top Operators'},axis=1)
    gb2 = pd.merge(gb2,gbop,on='bgCountyName',how='left')

#     gbprop = df[df.bgCAS=='proprietary'].groupby('bgCountyName',as_index=False).size()
# #     gbprop.bgCAS.fillna(0,inplace=True)
#     gbprop = gbprop.rename({'size':'Trade Secret records'},axis=1)
#     gb2 = pd.merge(gb2,gbprop,on='bgCountyName',how='left')
#     gb2['Trade Secret records'].fillna(0,inplace=True)
    
    gbtbwv = df.groupby(['bgCountyName','DisclosureId'],as_index=False)['TotalBaseWaterVolume'].first()
    gbtbwv = gbtbwv.groupby('bgCountyName',as_index=False)['TotalBaseWaterVolume'].sum().rename({'TotalBaseWaterVolume':'tot_gallons_water'},
                                                                                                axis=1)
    gbtbwv.tot_gallons_water = gbtbwv.tot_gallons_water.map(lambda x: th.round_sig(x,3))
    gb2 = pd.merge(gb2,gbtbwv,on='bgCountyName',how='left')

    #print(gb2.head())
    gb2['County'] = '<center><h4>'+gb2.bgCountyName.str.title().map(lambda x: th.getCountyLink(x,statename,x))+'</h4></center>'
    gb2 = gb2.drop('bgCountyName',axis=1)
    iShow(gb2.sort_values('disclosure_count',ascending=False)[['County','disclosure_count',
                                                               #'Trade Secret records'
                                                               'tot_gallons_water',
                                                               'Top Operators']].reset_index(drop=True),
         classes="display compact cell-border")
        


# Where are the fracking locations in this state?
This is not an exhaustive set of wells in these counties; it is only those wells for which the operating company submits a chemical disclosure to FracFocus.  In addition, this map omits disclosures for which location information is conflicting, such as the Latitude/Longitude values are outside of the reported county.

In [ ]:
CountyMap(alldf)

---
## County-based details

In [ ]:
CountyCntTable(alldf)

---
## Who are the Operators in this state?

In [ ]:
def make_water(row):
    s = str(th.round_sig(row.TotalBaseWaterVolume,3))
    s += '<br>'
    s += str(th.round_sig(row.TBWV90,3))
    return s

alldf['year'] = alldf.date.dt.year.astype(str)
gbOp = alldf.groupby(['DisclosureId','bgCountyName','bgOperatorName','year'],as_index=False)['TotalBaseWaterVolume'].first()
gbOp.bgCountyName = gbOp.bgCountyName.str.title()

gbOp1 = gbOp.groupby('bgOperatorName',as_index=False)['DisclosureId'].count().rename({'DisclosureId':'num_fracks'},axis=1)

gbOp2 = gbOp.groupby(['bgOperatorName','year'],as_index=False)['DisclosureId'].count()
gbOp2['year_cnt'] = gbOp2.year + '(' + gbOp2.DisclosureId.astype(str) + ')'

gbOpY = gbOp2.groupby('bgOperatorName')['year_cnt'].apply(set).reset_index()
gbOpY['years'] = gbOpY.year_cnt.map(lambda x: th.xlate_to_str(x,sep='<br>'))

gbOp3 = gbOp.groupby(['bgOperatorName','bgCountyName'],as_index=False)['DisclosureId'].count()
gbOp3['counties_cnt'] = gbOp3.bgCountyName + '(' + gbOp3.DisclosureId.astype(str) + ')'

gbOp4 = gbOp3.groupby('bgOperatorName')['counties_cnt'].apply(set).reset_index()
gbOp4['counties'] = gbOp4.counties_cnt.map(lambda x: th.xlate_to_str(x,sep='<br>'))

# gbOp5 = df.groupby('bgOperatorName')['OperatorName'].agg(lambda x: x.value_counts().index[0])
gbOp5 = alldf.groupby('bgOperatorName')['OperatorName'].apply(set).reset_index()
gbOp5['names'] = gbOp5.OperatorName.map(lambda x: th.xlate_to_str(x,sep='<br>'))
# gbOp5.names = gbOp5.names + '<br>[' + gbOp5.bgOperatorName + ']'

gbOp6 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].median()
gbOp6.rename({'TotalBaseWaterVolume':'Water,\nmedian (gal)'},axis=1,inplace=True)
# gbOp7 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].agg(lambda x: np.percentile(x,90))
gbOp7 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].max()
gbOp7.rename({'TotalBaseWaterVolume':'Water,\nmax (gal)'},axis=1,inplace=True)
mg = pd.merge(gbOp6,gbOp7,on='bgOperatorName')
# mg.fillna(0,inplace=True)
# mg['TBWV'] = mg.apply(lambda x: make_water(x),axis=1)
mg = pd.merge(mg,gbOp1,on='bgOperatorName')
mg = pd.merge(mg,gbOpY,on='bgOperatorName')
mg = pd.merge(mg,gbOp4[['bgOperatorName','counties']],on='bgOperatorName')
mg = pd.merge(mg,gbOp5,on='bgOperatorName').sort_values('num_fracks',ascending=False)

mg['link'] = mg.bgOperatorName.map(lambda x: th.getOpLink(x,x))
mg.names = mg.names + '<br>[' + mg.link + ']'

iShow(mg[['names','num_fracks','years','counties','Water,\nmedian (gal)','Water,\nmax (gal)']].reset_index(drop=True),
      maxBytes=0,columnDefs=[{"width": "150px", "targets": 0}])

In [ ]:
# from matplotlib.offsetbox import AnchoredText

# def proprietary_plot(df,plot_title='TEST',minyr=2011,maxyr=2021):
#     df = df.copy()
#     df['year'] = df.date.dt.year
#     df = df[df.year<=maxyr]
#     df = df[df.year>=minyr]
#     prop = df.is_proprietary
#     gb = df[prop].groupby('DisclosureId',as_index=False)['bgCAS'].count().rename({'bgCAS':'numprop'},axis=1)
#     gb1 = df[df.is_valid_cas].groupby('DisclosureId',as_index=False)['bgCAS'].count().rename({'bgCAS':'numvalid'},axis=1)
#     gb2 = df.groupby('DisclosureId',as_index=False)['date'].first()
#     mg = pd.merge(gb2,gb,on='DisclosureId',how='left')
#     mg = pd.merge(mg,gb1,on='DisclosureId',how='left')
#     mg.fillna(0,inplace=True) # there will be disclosures with 0 proprietary; need to fill
#     mg['percProp'] = (mg.numprop / mg.numvalid) * 100

#     mg['propCut'] = pd.cut(mg.percProp,right=False,bins=[0,0.0001,10,25,50,101],
#                           labels=['no proprietary claims','up to 10% proprietary claims',
#                                   'between 10 and 25% proprietary claims',
#                                   'between 25 and 50% proprietary claims',
#                                   'greater than 50% proprietary claims'])
#     mg['year'] = mg.date.dt.year
#     out = mg.drop(['date','DisclosureId'],axis=1)
#     t = out[out.numvalid>0].groupby(['year','propCut'],as_index=False)['numvalid'].count()
#     sums = t.groupby('year',as_index=False)['numvalid'].sum().rename({'numvalid':'tot'},axis=1)
#     t = pd.merge(t,sums,on='year',how='left')
#     t['PercentProp'] = t.numvalid/t.tot *100

#     piv = t.pivot(index='year', columns='propCut', values='PercentProp')

#     ax = piv.plot.area(figsize=(12,7),ylim=(0,100),xlim=(minyr,maxyr),colormap='Reds')
#     ax.set_title(f'Percentage of valid records that are Trade Secret claims at the disclosure level', fontsize=16)
#     handles, labels = ax.get_legend_handles_labels()
#     ax.legend(handles[::-1], labels[::-1], title='Disclosure Proprietary\nPercentage class\n',
#               loc='upper left',bbox_to_anchor=(1, 1))
#     ax.set_ylabel('Percentage of disclosures', fontsize=16)
#     ax.set_xlabel('Year', fontsize=16)
#     plt.xticks(fontsize=14)
#     plt.yticks(fontsize=14)
#     plt.suptitle(f'{plot_title}',fontsize=24)

#     gb = df.groupby(['year','DisclosureId'],as_index=False)['bgCAS'].count()
#     gb = gb.groupby('year',as_index=False)['DisclosureId'].count()#.rename({'DisclosureId':'number of disclosures'},axis=1)
#     s = 'Number of disclosures by year:\n\n'
#     for i,row in gb.iterrows():
#         s+= f'   {row.year}: {row.DisclosureId:7,} \n'
#     at2 = AnchoredText(s,
#                        loc='lower left', prop=dict(size=10), frameon=False,
#                        bbox_to_anchor=(1., 0.),
#                        bbox_transform=ax.transAxes
#                        )
#     at2.patch.set_boxstyle("square,pad=0.")
#     ax.add_artist(at2)

    
# # test = 'pennsylvania'
# # variable = 'bgStateName'
# testtitle = statename.title() +': Trade Secret frequency'
# if len(alldf.DisclosureId.unique())>500:
#     # small-number states look silly with this plot
#     proprietary_plot(alldf,testtitle,minyr=2011,maxyr=alldf.date.dt.year.max())


In [ ]:
# # wells_in_dist_fn = 'FFwells_in_school_districts.csv'
# distFF = pd.read_csv(wells_in_dist_fn)
# distFF = distFF[distFF.bgStateName==statename]
# distFF.num_FF_wells.fillna(0,inplace=True)
# iShow(distFF[['NAME','GEOID','num_FF_wells','num_all_wells']].reset_index(drop=True),
#      classes="display compact cell-border")